In [1]:
from libs import full_model
from libs import getData
from libs import dataAug

from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt

import tensorflow as tf
import numpy as np
import cv2 as cv
import os
import pickle

0.4.0


In [2]:
#global variables
lambda_value = 1 
patience = 4
k_fold = 10
seed = 12
lr = 0.001
my_adam = Adam(learning_rate=lr)

img_path = './dataset/Tissue images/'
label_path = './dataset/Annotations/'
data_ids = [i[:-4] for i in os.listdir(img_path)]

## get data

In [3]:
X_data, Y_data = getData.data(img_path, label_path, data_ids)

Reading image...


100%|██████████| 30/30 [00:00<00:00, 32.30it/s]

Done!
Generating point annotation...



 17%|█▋        | 5/30 [00:05<00:27,  1.10s/it]


KeyboardInterrupt: 

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(50, 30))
look_at = np.random.choice(30, 3, replace=False)

for j in range(3):
    axs[0][j].imshow(X_data[look_at[j]])
    axs[1][j].imshow(Y_data[look_at[j],:,:,2] + Y_data[look_at[j],:,:,1] + \
                Y_data[look_at[j],:,:,0], cmap='gray')

plt.show()

## data split

In [ ]:
#test data is never change.
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.1, random_state=seed)
X_test, y_test = (X_test.astype(np.float32))/255, y_test.astype(np.float32)

X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_train.dtype

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 8))

axs[0].imshow(X_test[0])
axs[1].imshow(X_test[1])
axs[2].imshow(X_test[2])

plt.show()

# 10 fold validation

In [ ]:
kf = KFold(n_splits=9)
train_indexs = []
val_indexs = []

for train_index, val_index in kf.split(X_train):
    train_indexs.append(train_index)
    val_indexs.append(val_index)

In [ ]:
i=1

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5)


cp_path = f'./old_model/{i}'
model_cp = callbacks.ModelCheckpoint(filepath=os.path.join(cp_path, 'fpn_w_attention.{val_loss:.4f}.h5'), \
                                     monitor='val_loss', save_best_only=True)
'''
------------------dataset split---------------------------
'''
print()
print(f'{i} validation')

X_train_new, y_train_new, X_val, y_val = X_train[train_indexs[i]], y_train[train_indexs[i]], \
                                            X_train[val_indexs[i]], y_train[val_indexs[i]]

print('after split: ',X_train_new.shape, X_train.shape)
X_train_new, y_train_new = dataAug.myDataAug(X_train_new, y_train_new, 4)

print('after aug:', X_train_new.shape, X_train.shape)

X_train_new, y_train_new, X_val, y_val = \
                X_train_new.astype(np.float32), y_train_new.astype(np.float32), \
                X_val.astype(np.float32), y_val.astype(np.float32)

X_train_new, X_val = X_train_new/255, X_val/255

'''
------------------training---------------------------
'''
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

#load model
model = full_model.get_model()
#model.summary()

losses = {
    'rs50_final_conv2_sigmoid': full_model.loss1,
    'second_term_sub' : full_model.loss2
}
model.compile(optimizer=my_adam, 
              loss=  losses , 
              metrics={'rs50_final_conv2_sigmoid': full_model.my_IoU})

history = model.fit(X_train_new, [y_train_new, y_train_new], validation_data=(X_val, y_val), batch_size=2, epochs=60, callbacks=[reduce_lr, model_cp])

'''
----------------------saving logs-----------------------
'''    
with open(os.path.join(cp_path, 'fpn_w_attention_history'), 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

# Show results

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(40, 20))

for i,j in emurator(axs.faltten()):
    j.imshow(X_data[look_at[i]])
    j.imshow(Y_data[look_at[i],:,:,2] + Y_data[look_at[i],:,:,1] + \
            Y_data[look_at[i],:,:,0], cmap='gray')

plt.show()

In [ ]:
look_at = np.random.choice(30, 3, replace=False)

re = model.predict([X_train[look_at]])


fig, axs = plt.subplots(1, 3, figsize=(20, 16))

axs[0].imshow(X_train[my_id])
axs[1].imshow(y_train[my_id,:,:,2].astype(np.uint8), cmap='gray')
axs[2].imshow(a, cmap='gray')

In [ ]:
my_id = 2
print(X_val[my_id].shape)
re = model.predict(np.array([X_val[my_id]]))
a = re[0][0,:,:,0]
a[a < 0.5] = 0
a[a != 0] = 1
print(np.sum(a))

fig, axs = plt.subplots(1, 3, figsize=(20, 16))

axs[0].imshow(X_val[my_id])
axs[1].imshow(y_val[my_id,:,:,2].astype(np.uint8), cmap='gray')
axs[2].imshow(a, cmap='gray')

In [ ]:
X_data[my_id].shape

In [ ]:
model.evaluate(X_test, y_test)